<a href="https://colab.research.google.com/github/ELehmann91/geo_sat_cli/blob/master/eo_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install enum
! pip install geopandas
! pip install shapely
! pip install lightgbm
! pip install eo-learn
! pip install sentinelhub

ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.
     |████████████████████████████████| 921kB 2.7MB/s 
     |████████████████████████████████| 11.9MB 40.2MB/s 
     |████████████████████████████████| 10.1MB 23.9MB/s 
  Created wheel for munch: filename=munch-2.3.2-py2.py3-none-any.whl size=6614 sha256=2d462f1423bca006767ea08da3b1c506ee964833a62249670ccc198afc659a3d
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch
  ERROR: Could not find a version that satisfies the requirement eolearn (from versions: none)
ERROR: No matching distribution found for eolearn
     |████████████████████████████████| 174kB 2.7MB/s 
     |████████████████████████████████| 133kB 47.8MB/s 
  Created wheel for sentinelhub: filename=sentinelhub-2.6.0-cp36-none-any.whl size=174968 sha256=3c42d314d5d70293a6defb4193c4db6478f7f4ab576326a3055d362dc0c85aa0
  Stored in direct

In [3]:
# Firstly, some necessary imports

# Jupyter notebook related
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Built-in modules
import pickle
import sys
import os
import datetime
import itertools
from enum import Enum

# Basics of Python data handling and visualization
import numpy as np
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Polygon
from tqdm import tqdm_notebook as tqdm

# Machine learning 
import lightgbm as lgb
from sklearn.externals import joblib
from sklearn import metrics
from sklearn import preprocessing

# Imports from eo-learn and sentinelhub-py
from eolearn.core import EOTask, EOPatch, LinearWorkflow, FeatureType, OverwritePermission, \
    LoadFromDisk, SaveToDisk, EOExecutor
from eolearn.io import S2L1CWCSInput, ExportToTiff
from eolearn.mask import AddCloudMaskTask, get_s2_pixel_cloud_detector, AddValidDataMaskTask
from eolearn.geometry import VectorToRaster, PointSamplingTask, ErosionTask
from eolearn.features import LinearInterpolation, SimpleFilterTask
from sentinelhub import BBoxSplitter, BBox, CRS, CustomUrlParam

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Part 1
1. Define the Area-of-Interest (AOI):
A geographical shape of Slovenia was taken from Natural Earth database and a buffer was applied. The shape is available in repository: example_data/svn_buffered.geojson
Convert it to selected CRS: taken to be the CRS of central UTM tile (UTM_33N)
Split it into smaller, manageable, non-overlapping rectangular tiles
Select a small 3x3 area for classification
Be sure that your choice of CRS is the same as the CRS of your reference data.

In the case that you are having problems with empty data being downloaded, try changing the CRS to something that suits the location of the AOI better.

### Get country boundary

In [0]:
#http://opendatalab.de/projects/geojson-utilities/

In [0]:
from shapely.geometry import Polygon

In [7]:
lat_point_list = [50.854457, 52.518172, 50.072651, 48.853033, 50.854457]
lon_point_list = [4.377184, 13.407759, 14.435935, 2.349553, 4.377184]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
print(polygon.geometry)

0    POLYGON ((4.37718 50.85446, 13.40776 52.51817,...
Name: geometry, dtype: geometry


In [8]:
# Folder where data for running the notebook is stored
#DATA_FOLDER = os.path.join('..', '..', 'example_data')

# Load geojson file
#country = gpd.read_file(os.path.join(DATA_FOLDER, 'svn_buffered.geojson'))

country = polygon
# Convert CRS to UTM_33N
country_crs = CRS.UTM_33N
country = country.to_crs(crs={'init': CRS.ogc_string(country_crs)})

# Get the country's shape in polygon format
country_shape = country.geometry.values.tolist()[-1]

# Plot country
country.plot()
plt.axis('off');

# Print size 
print('Dimension of the area is {0:.0f} x {1:.0f} m2'.format(country_shape.bounds[2] - country_shape.bounds[0],
                                                             country_shape.bounds[3] - country_shape.bounds[1]))

AttributeError: ignored